# Azure Document Intelligence Custom Template User Feedback Loop Experiment

This experiment demonstrates how to replicate the functionality of the [Azure AI Document Intelligence](https://learn.microsoft.com/en-GB/azure/ai-services/document-intelligence/overview) Studio custom model training process to showcase how to create a user feedback loop for improving the quality of document processing results.

This notebook showcases a more interactive user feedback experience, enabling a user to draw over an uploaded, analyzed document to provide feedback on the quality of results by highlighting incorrect or missing information with corrections. This implementation could be replicated in any client application using your chosen framework capabilities.

The goal is to showcase how a feedback mechanism can be implemented to allow the developers of custom models in Azure AI Document Intelligence to collect feedback from users to improve the model with the ability to retrain.

> **Note**: This notebook provides _one_ potential approach to user interaction, and can be interpreted in many ways based on your use case.

## Pre-requisites

> **Note**: Before continuing, please ensure that the [`Setup-Environment.ps1`](./Setup-Environment.ps1) script has been run to deploy the required infrastructure to Azure. This includes the Azure AI Document Intelligence resource and the Azure Storage account for creating a custom model.

This notebook uses [Dev Containers](https://code.visualstudio.com/docs/remote/containers) to ensure that all the required dependencies are available in a consistent local development environment.

The following are required to run this notebook:

- [Visual Studio Code](https://code.visualstudio.com/)
- [Docker Desktop](https://www.docker.com/products/docker-desktop)
- [Remote - Containers extension for Visual Studio Code](https://marketplace.visualstudio.com/items?itemName=ms-vscode-remote.remote-containers)

> **Note**: The Dev Container is pre-configured with the required dependencies and extensions. You can run this notebook outside of a Dev Container, but you will need to manually install the required dependencies including Poppler, Tesseract, and OpenCV.

The Dev Container will include the following dependencies by default:

- Debian 11 (Bullseye) base image
- Python 3.12
  - azure-ai-formrecognizer - for interacting with the Azure AI Document Intelligence service
  - azure-core - for interacting with the Azure AI Document Intelligence service
  - ipycanvas - for rendering the document and allowing the user to draw over it
  - ipykernel - for running the notebook
  - notebook - for running the notebook
  - opencv-python-headless - for image processing
  - pdf2image - for converting PDFs to images
  - pytesseract - for performing OCR on the document
- Poppler - used by pdf2image to convert PDFs to images
- Tesseract OCR - used by pytesseract to perform OCR on the document
- Python3 OpenCV - used for image processing

## Import Requirements

The following code block imports the required dependencies for this notebook.

It also configures the following:

- Setup the local working directory.
- Load local environment variables based on the output of the [`Setup-Environment.ps1`](./Setup-Environment.ps1) script run. The environment variables will be available in the [`.env`](./.env) file.
- Initialize the credential that will be used to authentication with the Azure services.

> **Note**: The [`Setup-Environment.ps1`](./Setup-Environment.ps1) script is not run as part of this notebook. It must be run separately, prior to running this notebook, to deploy the required infrastructure to Azure.

In [ ]:
import os
import json

from dotenv import dotenv_values
from ipywidgets import (VBox, Label)
from azure.identity import DefaultAzureCredential
from modules.app_settings import AppSettings
from modules.model_training_client import ModelTrainingClient
from modules.document_canvas import (DocumentCanvas)
from modules.document_intelligence_label import DocumentIntelligenceLabel
from modules.document_intelligence_result_formatter import DocumentIntelligenceResultFormatter

working_dir = os.path.abspath('')
settings = AppSettings(dotenv_values(f"{working_dir}/.env"))
azure_credential = DefaultAzureCredential(
    exclude_environment_credential=True,
    exclude_managed_identity_credential=True,
    exclude_shared_token_cache_credential=True,
    exclude_interactive_browser_credential=True,
    exclude_powershell_credential=True,
    exclude_visual_studio_code_credential=False,
    exclude_cli_credential=False
)

## Create a Document Intelligence Custom Model

In order to improve a model, you need to have one. This experiment comes prepared with the data required to train a custom model. The data is located in the [`model_training`](./model_training/) directory and contains a set of invoices that will be used by the following steps.

The steps below perform the following:

- Configures the initial values that will represent the model naming convention.
- Create a model training client (using the provided class) and run it to upload the files to Azure Blob Storage, and training the model using Azure AI Document Intelligence.

In [ ]:
# The name of the model
model_name = 'invoices' 

# The version of the model
initial_model_version = '1.0.0'

# The name of the model that will be registered in Azure AI Document Intelligence
initial_model_id = f"{model_name}-{initial_model_version}"

In [ ]:
model_training_client = ModelTrainingClient(settings=settings, use_azure_credential=False, azure_credential=azure_credential)

In [ ]:
# Resets the sample environment to only contain the initial training set. This is only necessary if the sample has been run previously.
model_training_client.delete_training_data("6")

# Uploads the initial training set to Azure Blob Storage and initiates model training using the uploaded data.
model_training_client.upload_training_data(f"{working_dir}/model_training")
invoice_model = model_training_client.create_model(model_name=initial_model_id)

## User Feedback Loop

The user feedback loop is a mechanism that allows users to provide feedback on the quality of the results generated by the model from interactions they have with it using their own data.

This section emulates what a user experience flow may present itself within an intelligent application interfacing with Azure AI Document Intelligence.

In [ ]:
# The name of the PDF file the user is providing.
pdf_file_name = 'Invoice_6.pdf'

# The directory containing the PDF file.
pdf_dir = os.path.join(working_dir, 'pdfs')

# The file path to the PDF file for loading.
pdf_path = os.path.join(pdf_dir, pdf_file_name)

# The file path to where the required JSON result from Azure AI Document Intelligence layout analysis will be stored.
pdf_ocr_path = os.path.join(pdf_dir, f"{pdf_file_name}.ocr.json")

# The file path to where the required JSON result for Azure AI Document Intelligence labels will be stored after user feedback.
pdf_labels_path = os.path.join(pdf_dir, f"{pdf_file_name}.labels.json")

### Run layout analysis on the PDF document using Azure AI Document Intelligence

This step will use the Azure AI Document Intelligence service to perform layout analysis on the PDF document. When complete, the files will be saved to the `./pdfs` directory with the name format `<pdf_file_name>.ocr.json`.

> **Note**: This specific step does not need to be run every time. The layout analysis is only required to be run once to capture the initial state of the document.

In [ ]:
model_training_client.run_layout_analysis(pdf_path, pdf_ocr_path, 'prebuilt-layout')

### Display the PDF in the notebook for user feedback

The following code will perform the following:

1. Load the PDF document and store each page as an image using pdf2image.
1. Display the rendered image using Canvas below as an interactive element in an output cell. **Note**: The image is rendered at the original size of the PDF page.
1. Allow you to draw label regions over the rendered image by clicking/holding, dragging, and releasing the mouse.

Below is an example of this interaction in action.

![Demonstration of canvas selection](./media/canvas-selection.gif)

As well as displaying the PDF, the required fields for the model are also displayed.

In [ ]:
fields_file_path = os.path.join(working_dir, 'model_training', 'fields.json')
with open(fields_file_path, 'r') as f:
    fields = json.load(f)

display_fields = [Label(value=f'{field['fieldKey']}') for field in fields['fields']]
display_container = VBox(display_fields)

print('Fields to capture')
display(display_container)

> **Note**: This simple demonstration below to load the PDF to a canvas does not allow drawn regions to be removed or edited once drawn. To start again, you will need to re-run the cell below.

In [ ]:
doc_canvas = DocumentCanvas(working_dir)

canvases = doc_canvas.load_pdf(pdf_path)
for canvas in canvases:
    display(canvas)

## Process the user feedback into Document Intelligence labels format

Once the user has drawn borders over the document to provide feedback, the following code will process the drawn borders into the labels JSON format used by the Azure AI Document Intelligence service. The files will be saved to the `./pdfs` directory with the name format `<pdf_file_name>.labels.json`.

In a real-world scenario, the labels JSON files could be loaded into a UI to allow the user to update the label names associated with the custom model, and then retrain the model using the updated labels and PDF documents. For the purposes of this experiment, these are rendered as UI inputs in the notebook.

### Render the user feedback as UI inputs

The following code will render the user feedback as UI inputs in the notebook as output. You can update the label names associated with the custom model.

Each square border previously drawn over the document will be rendered as a UI input in the notebook. The fields will be pre-populated, and you will be able to select the label from the available options for the model, and update the text associated with the label.

> **Note**: If the label option is an array, you will be provided with additional options to provide the row number and the column label.

In [ ]:
labels = [DocumentIntelligenceLabel(label_region, fields) for label_region in doc_canvas.label_regions]
    
for label in labels:
    display(label.render())

### Create the labels JSON file

Once the user has updated the label names and text associated with the drawn borders, the following code will create the labels JSON file in the format required by the Azure AI Document Intelligence service. The file will be saved to the `./pdfs` directory with the name format `<pdf_file_name>.labels.json`.

In [ ]:
DocumentIntelligenceResultFormatter.save_to_labels_json(labels, pdf_file_name, pdf_labels_path)

## Retrain the model using the updated labels and PDF documents

Once complete, the generated user feedback can be uploaded to the Azure Storage account and used to retrain the model using the Azure AI Document Intelligence service.

In [ ]:
# The version of the updated model, in this example, a minor change by adding a new training document.
updated_model_version = "1.1.0"

# The name of the model that will be registered in Azure AI Document Intelligence
updated_model_id = f"{model_name}-{updated_model_version}"

# Uploads the updated user feedback documents to Azure Blob Storage and initiates model training using both the existing and new data.
model_training_client.upload_training_data(pdf_dir)
updated_model = model_training_client.create_model(model_name=updated_model_id)